In [60]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from ipywidgets import interact, FloatSlider, fixed , Output
from IPython.display import display
from sklearn.linear_model import LinearRegression

In [68]:
# Load data
data_300 = pd.read_csv("300_deg_diff_sr.csv")
data_535 = pd.read_csv("535_deg_diff_sr.csv")
epsT_full_300 = np.array(data_300["Strain"], dtype=float)
epsT_full_535 = np.array(data_535["Strain"], dtype=float)
stresses_300 = [c for c in data_300.columns if c != "Strain"]
stresses_535 = [c for c in data_535.columns if c != "Strain"]
stress_matrix_300 = data_300[stresses_300].to_numpy()
stress_matrix_535 = data_535[stresses_535].to_numpy()
params_by_temp = {
    "300": dict(E=26000.0, k=40.0, K=220.0, n1=1.0, A=4.0, C=0.7, n2=1.5, B=180.0),
    "535": dict(E=26000.0, k=40.0, K=220.0, n1=1.0, A=4.0, C=0.7, n2=1.5, B=180.0)
}

In [75]:
def simulate_sigma_backward_euler(
    epsT: np.ndarray,
    SR: float,
    E: float, k: float, K: float, n1: float,
    A: float, C: float, n2: float,
    B: float,
    eps_p0: float = 0.0,
    rho0: float = 1e-4,
    rho_floor: float = 1e-12,
    n_sub: int = 100,
    newton_tol: float = 1e-10,
    newton_max: int = 25,
):
    """
    Backward Euler (implicit) on eps_p and rho.
    Uses local Newton iterations per substep.
    """

    epsT = np.asarray(epsT, dtype=float)
    N = len(epsT)

    sigma = np.zeros(N)
    eps_p = np.zeros(N)
    rho = np.zeros(N)
    R = np.zeros(N)

    eps_p[0] = eps_p0
    rho[0] = max(rho0, rho_floor)
    R[0] = B * np.sqrt(rho[0])
    sigma[0] = E * (epsT[0] - eps_p[0])

    for i in range(1, N):
        d_epsT = epsT[i] - epsT[i - 1]

        dt = d_epsT / SR
        dt_sub = dt / n_sub
        epsT_sub0 = epsT[i - 1]

        epp = eps_p[i - 1]
        r = rho[i - 1]

        for s in range(n_sub):
            epsT_s = epsT_sub0 + (s + 1) * (d_epsT / n_sub)

            # Newton initial guess = previous state
            epp_new = epp
            r_new = r

            for _ in range(newton_max):
                r_new = max(r_new, rho_floor)
                R_new = B * np.sqrt(r_new)
                sigma_new = E * (epsT_s - epp_new)

                drive = (sigma_new - R_new - k) / K
                epp_dot = max(drive, 0.0) ** n1
                r_dot = A * (1.0 - r_new) * epp_dot - C * (r_new ** n2)

                # residuals
                F1 = epp_new - epp - dt_sub * epp_dot
                F2 = r_new - r - dt_sub * r_dot

                if max(abs(F1), abs(F2)) < newton_tol:
                    break

                # numerical Jacobian (robust & simple)
                h = 1e-8

                # dF/d(epp)
                epp_p = epp_new + h
                sigma_p = E * (epsT_s - epp_p)
                drive_p = (sigma_p - R_new - k) / K
                epp_dot_p = max(drive_p, 0.0) ** n1
                F1_p = epp_p - epp - dt_sub * epp_dot_p
                dF1_depp = (F1_p - F1) / h

                r_dot_p = A * (1.0 - r_new) * epp_dot_p - C * (r_new ** n2)
                F2_p = r_new - r - dt_sub * r_dot_p
                dF2_depp = (F2_p - F2) / h

                # dF/d(r)
                r_p = r_new + h
                R_p = B * np.sqrt(max(r_p, rho_floor))
                sigma_p = E * (epsT_s - epp_new)
                drive_p = (sigma_p - R_p - k) / K
                epp_dot_r = max(drive_p, 0.0) ** n1

                r_dot_r = A * (1.0 - r_p) * epp_dot_r - C * (r_p ** n2)

                F1_r = epp_new - epp - dt_sub * epp_dot_r
                dF1_dr = (F1_r - F1) / h

                F2_r = r_p - r - dt_sub * r_dot_r
                dF2_dr = (F2_r - F2) / h

                # Solve 2x2 Newton system
                J = np.array([[dF1_depp, dF1_dr],
                              [dF2_depp, dF2_dr]])
                F = np.array([F1, F2])

                delta = np.linalg.solve(J, -F)
                epp_new += delta[0]
                r_new += delta[1]

            epp = epp_new
            r = max(r_new, rho_floor)

        eps_p[i] = epp
        rho[i] = r
        R[i] = B * np.sqrt(r)
        sigma[i] = E * (epsT[i] - epp)

    return sigma, eps_p, rho, R

In [ ]:
# Create an Output widget for the plot
output = Output()

# Interactive plotting function
def plot_simulation(temp, E, k, K, n2_0, n2_1, n2_2, A, C, n1, B, strains, stress_data, strain_data):
    n2_list = [n2_0, n2_1, n2_2]
    with output:
        output.clear_output(wait=True)  # Clear previous plot

        
        plt.figure(figsize=(8, 6))
        plt.title(f"Simulation and experiment plot for {temp}°C")
        for i in range(len(strains)):
            sigma_exp = stress_data[:, i]
            mask = ~np.isnan(sigma_exp)
            epsT = strain_data[mask]
            sigma_exp = sigma_exp[mask]

            sigma_model, _, _, _ = simulate_sigma_backward_euler(
                epsT=epsT,
                SR=strains[i],
                E=E, k=k, K=K, n1=n1, A=A, C=C, n2=n2_list[i], B=B,
                rho0=1e-4,
                n_sub=10
            )
            plt.plot(epsT, sigma_model, label=f'Model, sr = {strains[i]}')
            plt.scatter(epsT, sigma_exp, label=f'Experiment, sr = {strains[i]}', alpha=0.7, s=5)

        plt.xlabel('Strain')
        plt.ylabel('Stress (MPa)')
        plt.legend()
        plt.grid(True)
        plt.show()

# Create interactive widgets

E_slider = FloatSlider(min=1000, max=80000, step=500, value=30000, description='E (MPa):')
k_slider = FloatSlider(min=0, max=200, step=1, value=50, description='k (MPa):')
K_slider = FloatSlider(min=10, max=500, step=1, value=200, description='K (MPa):')
n2_0_1= FloatSlider(min=0.01, max=2, step=0.05, value=1.0, description='n2_sr_0_1:')
n2_1 = FloatSlider(min=0.01, max=2, step=0.05, value=1.0, description='n2_sr_1:')
n2_5 = FloatSlider(min=0.01, max=2, step=0.05, value=1.0, description='n2_sr_5:')
A_slider = FloatSlider(min=0.01, max=5, step=0.05, value=5.0, description='A:')
C_slider = FloatSlider(min=1, max=200, step=1, value=0.5, description='C:')
n1_slider = FloatSlider(min=0.1, max=5, step=0.1, value=1.5, description='n1:')
B_slider = FloatSlider(min=10, max=500, step=1, value=200, description='B (MPa):')

# Display the output widget
display(output)

strain_rates = [0.1, 1, 5]
# Use interact to create the interactive plot
interact(plot_simulation,
         temp = fixed(300),
         E=E_slider, 
         k=k_slider, 
         K=K_slider,
         n1=n1_slider, 
         A=A_slider, 
         C=C_slider, 
         n2_0=n2_0_1, n2_1=n2_1, n2_2=n2_5,
         B=B_slider, 
         strains=fixed(strain_rates),
         stress_data=fixed(stress_matrix_300),
         strain_data=fixed(epsT_full_300))

Output()

interactive(children=(FloatSlider(value=30000.0, description='E (MPa):', max=80000.0, min=1000.0, step=500.0),…

<function __main__.plot_simulation(temp, E, k, K, n2_0, n2_1, n2_2, A, C, n1, B, strains, stress_data, strain_data)>

# 535°C 

In [ ]:
# Create an Output widget for the plot
output = Output()

# Interactive plotting function
def plot_simulation(temp, E, k, K, n2_0, n2_1, n2_2, A, C, n1, B, strains, stress_data, strain_data):
    n2_list = [n2_0, n2_1, n2_2]
    with output:
        output.clear_output(wait=True)  # Clear previous plot

        
        plt.figure(figsize=(8, 6))
        plt.title(f"Simulation and experiment plot for {temp}°C")
        for i in range(len(strains)):
            sigma_exp = stress_data[:, i]
            mask = ~np.isnan(sigma_exp)
            epsT = strain_data[mask]
            sigma_exp = sigma_exp[mask]

            sigma_model, _, _, _ = simulate_sigma_backward_euler(
                epsT=epsT,
                SR=strains[i],
                E=E, k=k, K=K, n1=n1, A=A, C=C, n2=n2_list[i], B=B,
                rho0=1e-4,
                n_sub=10
            )
            plt.plot(epsT, sigma_model, label=f'Model, sr = {strains[i]}')
            plt.scatter(epsT, sigma_exp, label=f'Experiment, sr = {strains[i]}', alpha=0.7, s=5)

        plt.xlabel('Strain')
        plt.ylabel('Stress (MPa)')
        plt.legend()
        plt.grid(True)
        plt.show()

# Create interactive widgets

E_slider = FloatSlider(min=1000, max=80000, step=500, value=30000, description='E (MPa):')
k_slider = FloatSlider(min=0, max=200, step=1, value=50, description='k (MPa):')
K_slider = FloatSlider(min=10, max=500, step=1, value=200, description='K (MPa):')
n2_0_1= FloatSlider(min=0.01, max=2, step=0.05, value=1.0, description='n2_sr_0_1:')
n2_1 = FloatSlider(min=0.01, max=2, step=0.05, value=1.0, description='n2_sr_1:')
n2_5 = FloatSlider(min=0.01, max=2, step=0.05, value=1.0, description='n2_sr_5:')
A_slider = FloatSlider(min=0.01, max=5, step=0.05, value=5.0, description='A:')
C_slider = FloatSlider(min=1, max=200, step=1, value=0.5, description='C:')
n1_slider = FloatSlider(min=0.1, max=5, step=0.1, value=1.5, description='n1:')
B_slider = FloatSlider(min=10, max=500, step=1, value=200, description='B (MPa):')

# Display the output widget
display(output)

strain_rates = [0.1, 1, 5]
# Use interact to create the interactive plot
interact(plot_simulation,
         temp = fixed(535),
         E=E_slider, 
         k=k_slider, 
         K=K_slider,
         n1=n1_slider, 
         A=A_slider, 
         C=C_slider, 
         n2_0=n2_0_1, n2_1=n2_1, n2_2=n2_5,
         B=B_slider, 
         strains=fixed(strain_rates),
         stress_data=fixed(stress_matrix_535),
         strain_data=fixed(epsT_full_535))

Output()

interactive(children=(FloatSlider(value=30000.0, description='E (MPa):', max=80000.0, min=1000.0, step=500.0),…

<function __main__.plot_simulation(temp, E, k, K, n2_0, n2_1, n2_2, A, C, n1, B, strains, stress_data, strain_data)>